# GR5243 Group Project
##### Xingchen Ji, Yuting Wang, Hongyi Xu, and Jiacan Zhou

### Part 1. Data Collection

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
sns.set_style('darkgrid')
%matplotlib inline

In [ ]:
eta = pd.read_csv("../Data/RTA.csv")

In [ ]:
eta.head()

### Part 2. Data Cleaning

##### (a) Invalid Variable Dropping

In [ ]:
eta.isna().sum()

In [ ]:
eta.drop(["Service_year_of_vehicle", "Defect_of_vehicle", "Work_of_casuality", "Fitness_of_casuality"], axis = 1, inplace = True)
eta.drop(["Time", "Weather_conditions", "Casualty_class", "Sex_of_casualty", "Age_band_of_casualty", "Casualty_severity"], axis = 1, inplace = True)

In [ ]:
categorical = [col for col in eta.columns]
categorical.remove("Number_of_vehicles_involved")
categorical.remove("Number_of_casualties")
categorical.remove("Accident_severity")
numerical = ["Number_of_vehicles_involved", "Number_of_casualties"]

##### (b) Missing Data Cleaning

In [ ]:
eta.dropna(subset = categorical, inplace = True)

### Part 3. Explanatory Data Analysis

##### (a) Descriptive Statistics

In [ ]:
eta.info()

In [ ]:
eta.describe()

In [ ]:
eta["Accident_severity"].value_counts()
fig, ax = plt.subplots(1, 2, figsize = (15, 5))
eta["Accident_severity"].value_counts().plot.pie(ax = ax[0], autopct = "%.2f%%", title = "Pie Chart of Accident Severity")
sns.histplot(eta["Accident_severity"], ax = ax[1]).set(title = "Histogram of Accident Severity", xlabel = "Accident Severity")

##### (b) Data Visualization

In [ ]:
plt.scatter(x = "Number_of_vehicles_involved", y = "Number_of_casualties", data = eta)
plt.title("Scatterplot of Number of Vehicles Involved vs Number of Casualties")
plt.xlabel("Number of Vehicles Involved")
plt.ylabel("Number of Casualties");

In [ ]:
fig, ax = plt.subplots(7, 3, figsize = (30, 90))
for var, subplot in zip(categorical, ax.flatten()):
    sns.histplot(eta[var], ax = subplot).set(ylabel = "")
    subplot.set_title("Histogram of " + var, fontsize = 18)
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
    if len(var) > 5:
        subplot.set_xlabel(var, fontsize = 6)
    else:
        subplot.set_xlabel(var, fontsize = 12)
    if subplot != ax[-1, -1]:
        subplot.set_xlabel("")
plt.subplots_adjust(wspace = 0.2, hspace = 0.4)
fig.delaxes(ax[-1, -1])
fig.delaxes(ax[-1, -2])

##### (c) Correlation Analysis

In [ ]:
corr = eta.corr()
plt.figure(figsize = (9, 6))
sns.heatmap(corr, annot = True, cmap = "coolwarm").set(title = "Correlation Matrix");

### Part 4. Data Preprocessing

##### (a) Data Transformation

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
eta_fs = eta.copy()
eta_fs[numerical] = scaler.fit_transform(eta_fs[numerical])
y_cgan = eta_fs["Accident_severity"]

y_cgan = pd.get_dummies(y_cgan, columns = ["Accident_severity"])
y = y_cgan.to_numpy(dtype = np.float32)
X_cgan = pd.get_dummies(eta_fs, columns = categorical)
X_cgan = X_cgan.drop("Accident_severity", axis = 1)
X = X_cgan.to_numpy(dtype = np.float32)

##### (b) Data Augmentation by CGAN

In [ ]:
from CGAN_on_Scratch import *
dataset = TensorDataset(torch.tensor(X, dtype = torch.float32), torch.tensor(y, dtype = torch.float32))
dataloader = DataLoader(dataset, batch_size = 512, shuffle = True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
noise_dim = 100
label_dim = y_cgan.shape[1]
input_dim = X_cgan.shape[1]
num_numerical = 2
num_categorical = 156
best_params = bayesian_optimization_cgan(dataloader, device, noise_dim, label_dim, num_numerical, num_categorical)
print(f"Best Parameters: {best_params}")

In [ ]:
generator = Generator(noise_dim, label_dim, num_numerical, num_categorical, best_params["gen_hidden_dim"]).to(device)
discriminator = Discriminator(label_dim, num_numerical, num_categorical, best_params["disc_hidden_dim"]).to(device)
generator.apply(he_init)
discriminator.apply(he_init)
gen_optimizer = optim.Adam(generator.parameters(), lr = best_params["gen_lr"], betas = (best_params["gen_beta1"], 0.999))
disc_optimizer = optim.Adam(discriminator.parameters(), lr = best_params["disc_lr"], betas = (best_params["disc_beta1"], 0.999))
train_cgan(generator, discriminator, dataloader, device, gen_optimizer, disc_optimizer, best_params["num_epochs"], noise_dim)

In [ ]:
def generate_slight_injury_data(generator, num_data, noise_dim, label_dim, device):
    noise = torch.randn(num_data, noise_dim).to(device)
    labels = torch.zeros(num_data, label_dim).to(device)
    labels[:, 2] = 1
    fake_data = generator(noise, labels)
    return fake_data.detach().cpu()

def generate_serious_injury_data(generator, num_data, noise_dim, label_dim, device):
    noise = torch.randn(num_data, noise_dim).to(device)
    labels = torch.zeros(num_data, label_dim).to(device)
    labels[:, 1] = 1
    fake_data = generator(noise, labels)
    return fake_data.detach().cpu()

def generate_fatal_injury_data(generator, num_data, noise_dim, label_dim, device):
    noise = torch.randn(num_data, noise_dim).to(device)
    labels = torch.zeros(num_data, label_dim).to(device)
    labels[:, 0] = 1
    fake_data = generator(noise, labels)
    return fake_data.detach().cpu()

def round_one_hot(encoded_data, num_categories):
    start_index = 0
    for cate in num_categories:
        current_data = encoded_data[:, start_index:start_index + cate]
        max_indices = torch.argmax(current_data, dim = 1, keepdim = True)
        one_hot = torch.zeros_like(encoded_data)
        one_hot.scatter_(1, max_indices, 1)
        encoded_data[:, start_index:start_index + cate] = one_hot[:, start_index:start_index + cate]
        start_index += cate
    return encoded_data

num_categories = [7, 5, 3, 7, 4, 7, 17, 4, 13, 7, 9, 8, 5, 4, 4, 10, 13, 9, 20]
fake1_data = generate_serious_injury_data(generator, 4000, noise_dim, label_dim, device)
fake1_numerical = fake1_data[:, :num_numerical]
fake1_categorical = fake1_data[:, num_numerical:]
fake1_categorical = round_one_hot(fake1_categorical, num_categories)
fake1_data = np.concatenate((fake1_numerical, fake1_categorical), axis = 1)
fake1_data = pd.DataFrame(fake1_data, columns = X_cgan.columns)

fake2_data = generate_fatal_injury_data(generator, 3000, noise_dim, label_dim, device)
fake2_numerical = fake2_data[:, :num_numerical]
fake2_categorical = fake2_data[:, num_numerical:]
fake2_categorical = round_one_hot(fake2_categorical, num_categories)
fake2_data = np.concatenate((fake2_numerical, fake2_categorical), axis = 1)
fake2_data = pd.DataFrame(fake2_data, columns = X_cgan.columns)

In [ ]:
fake1_data.head()

In [ ]:
fake2_data.head()

In [ ]:
X_aug = pd.concat([X_cgan, fake1_data, fake2_data], axis = 0).reset_index(drop = True)
y_fake = np.zeros((7000, 3))
y_fake[:4000, 1] = 1
y_fake[4000:, 0] = 1
y_fake = pd.DataFrame(y_fake, columns = y_cgan.columns)
y_aug = pd.concat([y_cgan, y_fake], axis = 0).reset_index(drop = True)

In [ ]:
y_aug.value_counts()

##### (c) Dimensionality Reduction by PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X_aug)
pca_data = pca.transform(X_aug)
pca_data_var = pca.explained_variance_ratio_
plt.figure(figsize = (10, 6))
plt.plot(np.cumsum(pca_data_var), marker = "o", linestyle = "--")
plt.xlabel("Number of Components")
plt.xlim(0, 100)
plt.ylabel("Cumulative Explained Variance")
plt.title("Explained Variance vs Number of Components")
plt.show();

In [ ]:
threshold = 60
pca_denoise = PCA(n_components = threshold)
pca_denoise.fit(X_aug)
data_pca_denoised = pca_denoise.transform(X_aug)
X_denoised = pca_denoise.inverse_transform(data_pca_denoised)
X_denoised = pd.DataFrame(X_denoised, columns = X_aug.columns)

##### (d) Feature Selection by XGBoost

In [ ]:
y_xg = y_aug.values
y_xg = np.argmax(y_xg, axis = 1)
y_xg = pd.Series(y_xg)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_denoised, y_xg, test_size = 0.3, random_state = 233)

from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators = 100, max_depth = 3, learning_rate = 0.1, random_state = 233)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

feature_importance = xgb.feature_importances_
feature_importance = pd.DataFrame(feature_importance, index = X_train.columns, columns = ["importance"]).sort_values("importance", ascending = False)
aggregated_features = {}
variable_index = categorical + numerical
for feature in variable_index:
    for i in range(len(feature_importance.index)):
        if feature in feature_importance.index[i]:
            if feature in aggregated_features:
                aggregated_features[feature] += feature_importance.iloc[i, 0]
            else:
                aggregated_features[feature] = feature_importance.iloc[i, 0]
        else:
            if feature not in aggregated_features:
                aggregated_features[feature] = 0
aggregated_features = pd.DataFrame.from_dict(aggregated_features, orient = "index", columns = ["importance"]).sort_values("importance", ascending = False)
aggregated_features

In [ ]:
plt.figure(figsize = (10, 6))
sns.barplot(x = aggregated_features["importance"], y = aggregated_features.index).set(title = "Feature Importance", xlabel = "Importance", ylabel = "Features");

In [ ]:
drop_features = aggregated_features[aggregated_features["importance"] < 0.01].index.tolist()
drop_list = []
for feature in drop_features:
    for i in range(len(X_denoised.columns)):
        if feature in X_denoised.columns[i]:
            drop_list.append(X_denoised.columns[i])
X_denoised.drop(drop_list, axis = 1, inplace = True)

### Part 5. Data Spliting

In [ ]:
X_final = X_denoised.copy()
y_final = y_aug.copy()
y_final = y_final.values
y_final = np.argmax(y_final, axis = 1)
y_final = pd.Series(y_final)
X_final.to_csv("X_final.csv", index = False)
y_final.to_csv("y_final.csv", index = False)

In [ ]:
X = pd.read_csv("X_final.csv")
y = pd.read_csv("y_final.csv")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 233)

### Part 6. Model Selection
#### (a) SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
param_svc = {"C": [0.1, 1, 10, 100], "gamma": [1, 0.1, 0.01, 0.001], "kernel": ["rbf", "linear", "poly"], "degree": [1, 2, 3, 4]}
svc = GridSearchCV(SVC(), param_svc, refit = True, verbose = 0, n_jobs = 12)
svc.fit(X_train, y_train)
print(svc.best_params_)
print(svc.best_estimator_)
y_pred = svc.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, y_pred))
print("Classification report: ", classification_report(y_test, y_pred))
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square = True, annot = True, fmt = "d", cbar = False, xticklabels = ["Fatal", "Serious", "Slight"], yticklabels = ["Fatal", "Serious", "Slight"])
plt.xlabel("True Label")
plt.ylabel("Predicted Label");

#### (b) Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
param_gbc = {"learning_rate": [0.1, 0.01, 0.001], "n_estimators": [100, 200, 300], "max_depth": [1, 2, 3, 4, 5]}
gbc = GridSearchCV(GradientBoostingClassifier(), param_gbc, refit = True, verbose = 0, n_jobs = 12)
gbc.fit(X_train, y_train)
print(gbc.best_params_)
print(gbc.best_estimator_)
y_pred = gbc.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, y_pred))
print("Classification report: ", classification_report(y_test, y_pred))
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square = True, annot = True, fmt = "d", cbar = False, xticklabels = ["Fatal", "Serious", "Slight"], yticklabels = ["Fatal", "Serious", "Slight"])
plt.xlabel("True Label")
plt.ylabel("Predicted Label");

#### (c) Multilayer Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier
param_mlp = {"hidden_layer_sizes": [(128, 128), (128, 128, 128),(128, 128, 128, 128), (128, 128, 128, 128, 128)], "activation": ["relu", "logistic"], "solver": "adam", "alpha": [0.001, 0.01, 0.05, 0.1], "learning_rate": ["constant", "adaptive"]}
mlp = GridSearchCV(MLPClassifier(), param_mlp, refit = True, verbose = 0, n_jobs = 12)
mlp.fit(X_train, y_train)
print(mlp.best_params_)
print(mlp.best_estimator_)
y_pred = mlp.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, y_pred))
print("Classification report: ", classification_report(y_test, y_pred))
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square = True, annot = True, fmt = "d", cbar = False, xticklabels = ["Fatal", "Serious", "Slight"], yticklabels = ["Fatal", "Serious", "Slight"])
plt.xlabel("True label")
plt.ylabel("Predicted label");

### Part 7. Model Evaluation

In [ ]:
models = pd.DataFrame({
    "Model": ["Support Vector Machine", "Gradient Boosting", "Multilayer Perceptron"],
    "Accuracy": [accuracy_score(y_test, svc.predict(X_test)), accuracy_score(y_test, gbc.predict(X_test)), accuracy_score(y_test, mlp.predict(X_test))]})
models.sort_values(by = "Accuracy", ascending = False)

In [ ]:
ax = sns.barplot(x = "Model", y = "Accuracy", data = models.sort_values(by = "Accuracy", ascending = False))
for acc in ax.containers:
    ax.bar_label(acc, label_type = "center");